In [1]:
import graphlab

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] 1452172766 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/hzdy1994/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1452172766 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to gracedongy@gmail.com and will expire on December 27, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1470 - Server binary: /Users/hzdy1994/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452172766.log
[INFO] GraphLab Server Version: 1.7.1


In [3]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 
                  'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [12]:
def counting(word_count):
    if word in word_count:
        return word_count[word]
    else:
        return 0

In [13]:
for word in selected_words:
    products[word] = products['word_count'].apply(counting)

In [14]:
print products.head()

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
| Stop Pacifier Sucking with... | All of my kids have cried ... |  5.0   |
| Stop Pacifier Sucking with... | When the Binky Fairy came ... |  5.0   |
| A Tale of Baby's Days with... | Lovely book, it's bound ti... |  4.0   |
| Baby Tracker&reg; - Daily ... | Perfect for new parents. W... |  5.0   |
| Baby Tracker&reg; - Daily ... | A friend of mine pinned th... |  5.0   |
| Baby Tracker&reg; - Daily ... | This has been an easy way ... |  4.0   |
+------------------------

In [16]:
for word in selected_words:
    num = products[word].sum()
    print word,str(num)

awesome 2090
great 45206
fantastic 932
amazing 1363
love 42065
horrible 734
bad 3724
terrible 748
awful 383
wow 144
hate 1220


In [23]:
products = products[products['rating'] != 3]
products['sentiment'] = products['rating'] >=4
train_data,test_data = products.random_split(.8, seed=0)
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.245270     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.401960     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.559254     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [28]:
print selected_words_model['coefficients'].sort('value')

+-----------+-------+-------+------------------+
|    name   | index | class |      value       |
+-----------+-------+-------+------------------+
|  terrible |  None |   1   |  -2.09049998487  |
|  horrible |  None |   1   |  -1.99651800559  |
|   awful   |  None |   1   |  -1.76469955631  |
|    hate   |  None |   1   |  -1.40916406276  |
|    bad    |  None |   1   | -0.985827369929  |
|    wow    |  None |   1   | -0.0541450123333 |
|   great   |  None |   1   |  0.883937894898  |
| fantastic |  None |   1   |  0.891303090304  |
|  amazing  |  None |   1   |  0.892802422508  |
|  awesome  |  None |   1   |  1.05800888878   |
+-----------+-------+-------+------------------+
[12 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


In [29]:
coefficient = selected_words_model['coefficients'].sort('value')
print coefficient[0]
print coefficient[-1]

{'index': None, 'name': 'terrible', 'value': -2.090499984872607, 'class': 1}
{'index': None, 'name': 'love', 'value': 1.399898343017463, 'class': 1}


In [32]:
print selected_words_model.evaluate(test_data)

{'f1_score': 0.914242563530107, 'auc': 0.6648096413721418, 'recall': 0.9953531598513011, 'precision': 0.8453551912568306, 'log_loss': 0.4054747110365649, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+-----+-----+-------+------+
| threshold | fpr | tpr |   p   |  n   |
+-----------+-----+-----+-------+------+
|    0.0    | 1.0 | 1.0 | 27976 | 5328 |
|   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   2e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   3e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   4e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   5e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   6e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   7e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   8e-05   | 1.0 | 1.0 | 27976 | 5328 |
|   9e-05   | 1.0 | 1.0 | 27976 | 5328 |
+-----------+-----+-----+-------+------+
[100001 rows x 5 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns., 'confusion_matrix': C

In [36]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)
print sentiment_model.evaluate(test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.310641     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 4.340310     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 5.136925     | 0.92

In [43]:
#accuracy of the majority class classifier
positive = len(products[products['sentiment']==1])
total = len(products)
print 'accuracy of marjority class classifier is', float(positive)/total

accuracy of marjority class classifier is 0.841123344847


In [48]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment',ascending=False)
print diaper_champ_reviews

+-------------------------+-------------------------------+--------+
|           name          |             review            | rating |
+-------------------------+-------------------------------+--------+
| Baby Trend Diaper Champ | Baby Luke can turn a clean... |  5.0   |
| Baby Trend Diaper Champ | I LOOOVE this diaper pail!... |  5.0   |
| Baby Trend Diaper Champ | We researched all of the d... |  4.0   |
| Baby Trend Diaper Champ | My baby is now 8 months an... |  5.0   |
| Baby Trend Diaper Champ | This is absolutely, by far... |  5.0   |
| Baby Trend Diaper Champ | Diaper Champ or Diaper Gen... |  5.0   |
| Baby Trend Diaper Champ | Wow!  This is fabulous.  I... |  5.0   |
| Baby Trend Diaper Champ | I originally put this item... |  5.0   |
| Baby Trend Diaper Champ | Two girlfriends and two fa... |  5.0   |
| Baby Trend Diaper Champ | I am one of those super-cr... |  5.0   |
+-------------------------+-------------------------------+--------+
+-------------------------------+-

In [54]:
best_review = diaper_champ_reviews[0]
selected_words_model.predict(best_review, output_type='probability')

dtype: float
Rows: 1
[0.7969408512906712]

In [50]:
print diaper_champ_reviews[0]['review']

Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend's house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn't any added bulk around the sealing edge of the champ.
